In [24]:
import numpy as np
import librosa
import google.generativeai as genai
import yaml
import time

In [26]:
og_arr, og_sr = librosa.load("data/backyard-specs.m4a")
og_arr

/tmp/ipykernel_209950/3759619369.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  og_arr, og_sr = librosa.load("data/backyard-specs.m4a")
/home/kashyab/.pyenv/versions/3.11.8/envs/exp/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -3.4807206e-05,  2.6605767e-04,  3.1566259e-04], dtype=float32)

In [28]:
freq_arr = librosa.stft(og_arr)

### zero shot audio detection
* [clap model](https://huggingface.co/docs/transformers/model_doc/clap): it requires a text input which it yields a probability for. Perhaps get the set of species based on location and then use the model to tell us which one it could be? But the training set would require to have the insect sounds. Dataset of insect noises then to train on/
* just into a multimodal llm?

In [15]:
with open("config.yaml", "r") as conf_file:
    config = yaml.safe_load(conf_file)

In [42]:
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

In [49]:
audio_file = upload_to_gemini("data/test-capabilities.m4a", mime_type="audio/x-mp3")

Uploaded file 'test-capabilities.m4a' as: https://generativelanguage.googleapis.com/v1beta/files/cqtdvcpzlq7v


In [50]:
genai.configure(api_key=config["GEMINI_KEY"])
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")
prompt = "What kinds of insects do you hear?"
response = model.generate_content([prompt, audio_file])
print(response.text)

This audio does not contain the sounds of any insects. It sounds like a recording of someone walking on gravel or rocks. 
